In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import findspark
findspark.init()

import collections
import numpy as np
import pandas as pd
import datetime as dt
import os.path as osp
import matplotlib.pylab as plt
import seaborn as sns
import sklearn.preprocessing as prep
import sklearn.feature_selection as sfsel
import sklearn.tree as stree
import sklearn.ensemble as sensemble

from pyspark import keyword_only
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
sess = SparkSession.builder \
                   .master("local[*]") \
                   .config("spark.driver.memory", "32g") \
                   .getOrCreate()

In [5]:
test_df = sess.read.parquet("./stgt_sensors_with_date_geo_dwd_series.parquet/")
test_df.printSchema()
print(test_df.count())

root
 |-- hash: string (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- weekend: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- holiday: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- day_of_week: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- day_num: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- time_num: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- temperature: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- dwd_temperature: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- humidity: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- dwd_precipitation_height: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- pressure: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- dwd_wind_spe

In [8]:
test_df.withColumn("l", length("hash")).select("l").distinct().toPandas()

,l
0,12


In [19]:
test_df.columns

['hash',
 'start_year',
 'weekend',
 'holiday',
 'day_of_week',
 'day_num',
 'time_num',
 'temperature',
 'dwd_temperature',
 'humidity',
 'dwd_precipitation_height',
 'pressure',
 'dwd_wind_speed',
 'dwd_wind_direction',
 'dwd_clouds',
 'dwd_sun',
 'P1',
 'P2']

In [36]:
test_df.withColumn("hash_bin", substring("hash", 0, 5)) \
       .groupBy("hash_bin") \
       .agg(
         max('start_year').alias("start_year"),
         collect_list('weekend').alias('weekend'),
         collect_list('holiday').alias('holiday'),
         collect_list('day_num').alias('day_num'),
         collect_list('temperature').alias("temperature"),
         collect_list('dwd_temperature').alias("dwd_temperature"),
         collect_list('humidity').alias("humidity"),
         collect_list('dwd_precipitation_height').alias("dwd_precipitation_height"),
         collect_list('pressure').alias("pressure"),
         collect_list('dwd_wind_speed').alias("dwd_wind_speed"),
         collect_list('dwd_wind_direction').alias("dwd_wind_direction"),
         collect_list('dwd_clouds').alias("dwd_clouds"),
         collect_list('dwd_sun').alias("dwd_sun"),
         collect_list('P1').alias("P1"),
         collect_list('P2').alias("P2")
        ) \
        .limit(1).toPandas()

,hash_bin,start_year,weekend,holiday,day_num,temperature,dwd_temperature,humidity,dwd_precipitation_height,pressure,dwd_wind_speed,dwd_wind_direction,dwd_clouds,dwd_sun,P1,P2
0,u0wsj,2017,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.6495535714285714, 0.6510416666666669, 0.65...","[[25.166666666666668, 25.8, 26.183333333333337...","[[21.899999999999995, 21.900000000000002, 21.9...","[[40.800000000000004, 40.15, 38.5, 39.31666666...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[3.7000000000000006, 3.6999999999999997, 3.69...","[[260.0, 260.0, 260.0, 260.0, 250.0, 250.0, 25...","[[7.0, 7.0, 7.0, 7.0, 6.0, 6.0, 6.0, 6.0, 3.0,...","[[37.0, 37.0, 37.0, 37.0, 47.0, 47.0, 47.0, 47...","[[17.776666666666667, 26.673333333333336, 11.6...","[[10.623333333333335, 12.594999999999999, 4.87..."


In [33]:
test_df.toPandas()

,hash,start_year,weekend,holiday,day_of_week,day_num,time_num,temperature,dwd_temperature,humidity,dwd_precipitation_height,pressure,dwd_wind_speed,dwd_wind_direction,dwd_clouds,dwd_sun,P1,P2
0,u0wsyx2s1j08,2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0.28199404761904756, 0.28348214285714285, 0.2...","[0.9739583333333335, 0.984375, 0.9947916666666...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[8.200000000000001, 8.200000000000001, 8.20000...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[4.3999999999999995, 4.3999999999999995, 4.399...","[100.0, 100.0, 100.0, 130.0, 130.0, 130.0, 130...","[7.0, 7.0, 7.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[27.560833333333335, 32.675000000000004, 43.64...","[18.695833333333336, 20.548571428571428, 23.77..."
1,u0wt2ryn72xj,2016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0.671875, 0.673363095238089, 0.67485119047619...","[0.703125, 0.7135416666666732, 0.7239583333333...","[22.5, 22.699999999999832, 22.819230769230742,...","[20.299999999999972, 20.2000000000002, 20.2000...","[60.86666666666662, 60.73000000000012, 59.6576...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.0999999999999992, 1.1000000000000132, 1.100...","[170.0, 150.0, 150.0, 150.0, 150.0, 110.0, 110...","[8.0, 8.0, 8.0, 8.0, 8.0, 6.0, 6.0, 6.0, 6.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 20.0, 20.0, 20.0, 20...","[9.380000000000003, 6.920476190476227, 7.26222...","[6.405000000000006, 5.202380952380942, 5.06888..."
2,u0wt3q71zdcj,2016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, ...","[0.6436011904761901, 0.6450892857142856, 0.646...","[0.5052083333333336, 0.515625, 0.5260416666666...","[5.699999999999998, 5.649999999999996, 5.80555...","[3.5, 3.5, 3.5, 3.5, 3.7999999999999763, 3.799...","[44.799999999999976, 44.56250000000021, 44.766...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[3.0999999999999988, 3.1000000000000143, 3.099...","[130.0, 130.0, 130.0, 130.0, 110.0, 110.0, 110...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 7.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.0,...","[nan, 37.55111111111111, 39.41631578947387, 39...","[nan, 17.054444444444496, 17.901052631578988, ..."
3,u0wt5xnbyfmh,2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0.3117559523809524, 0.31324404761904756, 0.31...","[0.18229166666666666, 0.19270833333333334, 0.2...","[20.0, 14.484615384615381, 5.8066666666666675,...","[3.5, 3.5, 3.5, 3.1999999999999993, 3.2, 3.2, ...","[40.199999999999996, 43.14615384615385, 73.26,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.4000000000000001, 1.4, 1.3999999999999997, ...","[190.0, 190.0, 190.0, 160.0, 160.0, 160.0, 160...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.8883333333333336, 11.128461538461538, 14.90...","[2.686666666666667, 9.886923076923077, 12.826,..."
4,u0wkx07nkxj7,2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.10193452380952382, 0.10342261904761905, 0.1...","[0.7135416666666667, 0.7239583333333333, 0.734...","[28.028571428571432, 27.000000000000004, 22.25...","[14.0, 14.0, 14.0, 14.0, 13.5, 13.5, 13.5, 13....","[36.25714285714